In [1]:
import cv2 as cv
import numpy as np
import pandas as pd
import os

Liên kết đến Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Đường dẫn đến thư mục chứa các ảnh

In [3]:
images_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/Training'

Lưu trữ đặc trưng và nhãn

In [4]:
features = []
labels = []

Duyệt và trích xuất đặc trưng

In [5]:
for class_name in os.listdir(images_dir):
  class_dir = os.path.join(images_dir, class_name)

  if not os.path.isdir(class_dir):
    continue

  for image_name in os.listdir(class_dir):
    image_path = os.path.join(class_dir, image_name)

    image = cv.imread(image_path)
    if image is None:
      continue

    # HSV convert
    hsv_converted_img = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    # HUE mean, SAT mean
    hue_mean = np.mean(hsv_converted_img[:, :, 0])
    sat_mean = np.mean(hsv_converted_img[:, :, 1])

    # Find contour
    gray_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    _, thresh = cv.threshold(gray_img, 127, 255, cv.THRESH_BINARY)
    contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if contours:
      # Get max contour
      largest_contour = max(contours, key=cv.contourArea)
      contour_area = cv.contourArea(largest_contour)
      x, y, w, h = cv.boundingRect(largest_contour)
      aspect_ratio = float(w) / h if h != 0 else 0
    else:
      contour_area = 0
      aspect_ratio = 0

    # Save to arrays
    feature_vector = [hue_mean, sat_mean, contour_area, aspect_ratio]
    features.append(feature_vector)
    labels.append(class_name)

# Tạo DataFrame và lưu vào CSV
dataframe = pd.DataFrame(
    features,
    columns=[
        "Hue",
        "Saturation",
        "Contour Area",
        "Aspect Ratio"
    ]
)

dataframe["Label"] = labels
dataframe.to_csv(
    "/content/drive/MyDrive/Colab Notebooks/SVM Training/features.csv",
    index=False
)
print("Đã tạo file CSV: fruit_features.csv")


Đã tạo file CSV: fruit_features.csv
